# Model training

In [52]:
import pandas as pd

In [53]:
df = pd.read_csv("data/gemstone.csv")
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [54]:
df = df.drop(labels='id',axis=1)
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


## Independent and dependent variables

In [55]:
X = df.drop(labels=['price'],axis=1)
y = df[['price']]
X.head()
y.head()

,price
0,13619
1,13387
2,2772
3,666
4,14453


In [56]:
# Define which columns should be ordinal-encoded and which should be scaled
categorical_cols=X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [57]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair','Good','Very Good','Premium','Ideal']
color_categories = ['D','E','F','G','H','I','J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

## Handling missing values

In [58]:
df.isna().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
x          0
y          0
z          0
price      0
dtype: int64

In [59]:
from sklearn.impute import SimpleImputer # handling missing values
from sklearn.preprocessing import StandardScaler # handling features scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [60]:
## Numerical Pipeline
num_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)
## Categorical pipeline
categorical_pipeline= Pipeline(
    steps = [
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)
preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('categorical_pipeline',categorical_pipeline,categorical_cols)
])

## Train Test split

In [61]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=30,random_state=42)

In [62]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [63]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,categorical_pipeline__cut,categorical_pipeline__color,categorical_pipeline__clarity
0,-0.628272,0.997864,1.966018,-0.599701,-0.653255,-0.543245,-0.132938,-0.934081,0.016587
1,0.473999,2.292142,0.923694,0.536035,0.571430,0.777665,-2.144679,-0.934081,0.016587
2,-0.174396,-0.296414,-0.639793,-0.022820,0.018054,-0.035203,0.872933,-1.550201,0.016587
3,-0.109557,-1.220898,0.402531,0.094359,0.126915,0.008344,-1.138808,0.298158,0.016587
4,1.554657,-0.573759,-0.118631,1.572619,1.551179,1.474409,0.872933,-0.934081,0.016587


## Model training

In [64]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [65]:
regression = LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [66]:
regression.coef_

array([[ 6433.19426055,  -129.04433368,   -68.56968206, -1783.50010886,
         -392.69821755,  -103.47167526,    70.85885105,  -465.07115316,
          650.78455644]])

In [67]:
regression.intercept_

array([3969.21878859])

In [68]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [70]:
## train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
    }

model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    # Make Predictions
    y_pred = model.predict(X_test)
    mae, rmse, r2_square = evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Process')
    print('RMSE',rmse)
    print('MAE',mae)
    print('R2_Square',r2_square*100)

    print('-'*40)
    print('\n')

LinearRegression
Model Training Process
RMSE 875.0913960442674
MAE 617.8148753858678
R2_Square 94.58310906302275
----------------------------------------


Lasso
Model Training Process
RMSE 866.5533314402913
MAE 612.8770282352119
R2_Square 94.6882961117015
----------------------------------------


Ridge
Model Training Process
RMSE 875.0127344951084
MAE 617.7947254969222
R2_Square 94.58408286275073
----------------------------------------


ElasticNet
Model Training Process
RMSE 1073.3536788768163
MAE 847.4936658894953
R2_Square 91.85053756417865
----------------------------------------




In [71]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']